# Whole-slide image processing
이번 실습에서는 Histopathology 이미지가 저장된 형태인 Whole-slide image (WSI) 분석을 수행해본다.

In [ ]:
import math
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import PIL
import openslide
import cv2

In [ ]:
data_root_dir = "/datasets/CAMELYON16"
slide_name = "test_001"
slide_filepath = Path(data_root_dir) / f"testing/images/{slide_name}.tif"
annotation_filepath = Path(data_root_dir) / f"testing/lesion_annotations/{slide_name}.xml"

## OpenSlide
OpenSlide는 Whole-slide image 처리를 도와주는 라이브러리이다. 이를 이용하여 Whole slide 이미지를 읽어보자.

In [ ]:
print(f"Opening Slide {slide_filepath}")
slide = openslide.open_slide(slide_filepath)

In [ ]:
def print_slide_info(slide, slide_filepath):
    print("Level count: %d" % slide.level_count)
    print("Level dimensions: " + str(slide.level_dimensions))
    print("Level downsamples: " + str(slide.level_downsamples))
    print("Slide dimensions (width, height): " + str(slide.dimensions))
    print("Format: " + str(slide.detect_format(slide_filepath)))
    print("Properties:")
    for prop_key in slide.properties.keys():
        print("  Property: " + str(prop_key) + ", value: " + str(slide.properties.get(prop_key)))

In [ ]:
print_slide_info(slide, slide_filepath)

Whole slide image 파일은 아래와 같이 다양한 해상도의 이미지를 하나의 파일에 모두 포함하고 있습니다..

<img src="resources/svs.jpg" width="400" align="middle"/>

이 이미지에는 총 9의 `level` 이미지가 존재합니다

- `Level[0]`: 원본 이미지(Whole slide)를 그대로 담고 있으며, `downsample = 1`이다
- `Level[1]`: 원본 이미지를 `downsample = 2`로 다운 샘플링을 한 이미지가 보관되어 있다.
- `Level[2]`: 원본 이미지를 `downsample = 4`로 다운 샘플링을 한 이미지가 보관되어 있다.
- ...
- `Level[8]`: 가장 낮은 해상도의 이미지, `downsample = 256`, 이미지 크기 `(336, 350)`



`slide.read_region((x, y), level, (width, height))`함수를 이용하여 원하는 영역의 이미지를 읽어올 수 있습니다
- location (tuple) – `level = 0`에서의 left-top 픽셀 위치를 지정하는 튜플 `(x, y)`
- level (int) – the level number
- size (tuple) – 특정 `level`에서의 영역 크기`(width, height)`



In [ ]:
def slide_to_scaled_pil_image(slide, level):
    """
    Obtain scaled-down PIL image from WSI slide
    """
    
    slide_width, slide_height = slide.dimensions
    image = slide.read_region((0, 0), level, slide.level_dimensions[level]).convert("RGB")

    print(f"Origial slide size (width, height) : {slide_width}, {slide_height}")
    print(f"PIL Image size at level {level} : {image.size}")
    print(f"NumPy array shape at level {level} : {np.array(image).shape}")

    return image

(참고)

`OpenSlide`, `PIL`에서는 이미지 차원을 (width, height) 순으로 표현하고,

`OpenCV`, `np.array` 에서는 이미지 차원을 (height, width, channels) 순으로 표현합니다.

In [ ]:
pil_img = slide_to_scaled_pil_image(slide, level = 4)

plt.figure(figsize=(8,8))
plt.imshow(pil_img)
plt.show()

## Tumor Region annotation
Whole-slide 이미지에서 종양 영역(tumor region)을 정의하는 어노테이션(annotation)파일을 읽어보자.

In [ ]:
from xml.etree.ElementTree import parse

def parse_xml_annotations(annotation_filepath, mask_level):
    """
    Parses the XML annotation file to extract tumor area coordinates.

    Args:
        annotation_filepath (str): Path to the XML annotation file.
        mask_level (int): The level of the mask (resolution level).

    Returns:
        list: A list of coordinate lists for each annotated tumor area.
    """
    xml_root = parse(annotation_filepath).getroot()
    tumor_areas = []

    for region in xml_root.iter('Coordinates'):
        coordinates = []
        for point in region:
            x = round(float(point.get('X')) / (2 ** mask_level))
            y = round(float(point.get('Y')) / (2 ** mask_level))
            coordinates.append([x, y])
        tumor_areas.append(coordinates)

    return tumor_areas

- Tumor region은 `(x, y)` 좌표들의 집합으로 표현되며, 이 점들이 이루는 polygon내부가 tumor영역을 나타냅니다.
- `mask_level`값은 슬라이드 이미지의 어느 `level`에 대응하는 `(x ,y)` 좌표들을 담고 있을지 결정합니다.
- 일반적으로 mask image는 원본 이미지 (`level = 0`)보다 낮은 `level`에서 작업하여 계산 효율을 높입니다.

In [ ]:
tumor_areas = parse_xml_annotations(annotation_filepath, mask_level = 0)
print(f"Number of areas : {len(tumor_areas)}")
print(f"Tumor area #1 shape: {np.array(tumor_areas[0]).shape}")
print(f"Tumor area #1 values: {np.array(tumor_areas[0])}")

In [ ]:
def mark_tumor_area(slide_thumbnail, mask_level):
    """
    Draw contours of tumor area on given slide_thumbnail with same level
    """
    tumor_areas = parse_xml_annotations(annotation_filepath, mask_level)
    
    for area in tumor_areas:
        cv2.drawContours(image=slide_thumbnail, contours=np.array([area]),
                         contourIdx=-1, color=(0, 255, 0), thickness=4)
        
mask_level = 4
slide_thumbnail = np.array(slide_to_scaled_pil_image(slide, level = mask_level))
mark_tumor_area(slide_thumbnail, mask_level = mask_level)

plt.figure(figsize=(8,8))
plt.imshow(slide_thumbnail)
plt.show()

## Mask image 생성
Whole-slide image분석을 위해서는 Tumor mask와 Normal mask 두개의 마스크 이미지가 필요합니다.

<img src="resources/masks.png" width="600" align="middle"/>

### Tumor mask
Tumor영역을 나타내는 0, 255의 값을 가지는 이미지입니다.

XML 파일에서 읽은 종양 영역 annotation을 이용하여 그릴 수 있습니다.
- `np.zeros`를 이용하여 `mask_level`에 해당하는 마스크 이미지를 초기화 (`dtype=np.uint8`)
- 예를들어 특정 `level`에서의 slide 이미지의 크기가 `(width, height) = (100, 200)`이라면 `(200, 100)`의 모양(shape)을 가지는 `np.array` 생성한다.
- `cv2.drawContours`함수 ([docs](https://opencv-python.readthedocs.io/en/latest/doc/15.imageContours/imageContours.html))를 이용하여 tumor영역에 `color = 255`값을 채워준다. (`thickness = -1`으로 컨투어 내부를 채워줍니다)

### Tissue mask
Tissue mask는 slide image중 실제로 조직이 포함된 영역을 표현해줍니다.

이번 실습에서는 OpenCV의 Otsu's thresholding 알고리즘을 사용하여 tissue mask를 생성합니다.

<img src="resources/threshold.png" width="400" align="middle"/>

<img src="resources/Otsu.jpg" width="400" align="middle"/>

### Normal mask
Normal 조직이 포함된 영역을 표현해주는 0과 255의 값을 가지는 이미지입니다.
- `cv2.subtract`함수를 이용하여 Tissue mask에서 Tumor 영역을 빼주면 Normal mask를 얻을 수 있습니다.

### <mark>실습</mark> create_masks
위 설명을 참고하여 함수 `create_masks`를 완성하세요

In [ ]:
def create_masks(slide_filepath, annotation_filepath, mask_level):
    """
    Creates tumor, normal, and tissue masks using the XML annotations and Otsu's thresholding.

    Args:
        slide_filepath (str): Path to the slide image file.
        annotation_filepath (str): Path to the XML annotation file.
        mask_level (int): The level of the mask (resolution level).
    """

    # Load the slide at the specified level
    slide = openslide.OpenSlide(slide_filepath)
    slide_thumbnail  = np.array(slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level]).convert("RGB"))

    # Parse XML annotations to get tumor coordinates
    tumor_areas = parse_xml_annotations(annotation_filepath, mask_level)
    
    # Draw tumor boundaries on the slide thumbnail
    for area in tumor_areas:
        cv2.drawContours(image=slide_thumbnail, contours=np.array([area]),
                         contourIdx=-1, color=(0, 255, 0), thickness=4)
    
    ##### YOUR CODE START #####
    # Initialize empty mask for tumor mask
    tumor_mask = None # TODO
    # Fill the tumor areas on the tumor mask

    ##### YOUR CODE END #####

    # Create the tissue mask using Otsu's thresholding on the saturation channel
    slide_region = slide.read_region((0, 0), mask_level, slide.level_dimensions[mask_level])
    slide_rgb = cv2.cvtColor(np.array(slide_region), cv2.COLOR_RGBA2RGB)
    slide_hsv = cv2.cvtColor(slide_rgb, cv2.COLOR_RGB2HSV)
    saturation_channel = slide_hsv[:, :, 1] # 채도(Saturation) refers to intensity of colors. Tissue regions are likely to show higher saturation values
    _, tissue_mask = cv2.threshold(saturation_channel, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    ##### YOUR CODE START #####
    # Create the normal mask by excluding tumor areas from the tissue mask
    normal_mask = None # TODO
    ##### YOUR CODE END #####

    return slide, slide_thumbnail, tumor_mask, tissue_mask, normal_mask

In [ ]:
mask_level = 4  # define resolution level of mask

slide, slide_thumbnail, tumor_mask, tissue_mask, normal_mask = create_masks(slide_filepath, annotation_filepath, mask_level = mask_level)
print("Mask shape: ", tumor_mask.shape)

assert len(tumor_mask.shape) == 2, "Tumor mask should be single channel image"
assert slide_thumbnail.shape[:2] == tumor_mask.shape, "Tumor mask shape is inconsistant with slide image"
assert tumor_mask.shape == normal_mask.shape, "Normal mask shape is inconsistant with Tumor mask"

In [ ]:
def draw_masks(slide_thumbnail, tumor_mask, tissue_mask, normal_mask):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    images = [slide_thumbnail, tumor_mask, tissue_mask, normal_mask]
    titles = ['Slide Thumbnail', 'Tumor Mask', 'Tissue Mask', 'Normal Mask']

    for ax, img, title in zip(axes.ravel(), images, titles):
        cmap = 'gray' if len(img.shape) == 2 else None  # Use grayscale colormap for 2D masks
        ax.imshow(img, cmap=cmap, vmin=0, vmax=255)
        ax.set_title(title)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
draw_masks(slide_thumbnail, tumor_mask, tissue_mask, normal_mask)

## Patch extraction
Whole slide image는 크기가 매우 크므로 (주로 gigapixel) 딥러닝에 활용하기 위해서는 패치(patch)라 불리는 적당한 크기의 (예: 256x256 픽셀) 이미지로 분할하는 과정이 필요합니다.

<img src="resources/patches.png" width="400" align="middle"/>

- patch는 `level = 0`의 원본 이미지에서 추출합니다.
- 해당 patch가 tumor에 해당되는지 normal에 해당되는지 여부는 각각 이에 대응하는 `tumor_mask`와 `normal_mask`의 픽셀 값을 이용하여 판단합니다.
- 이때, mask 이미지와 원본 이미지의 해상도가 다르다는 것에 주의해야 합니다 (아래 이미지 참고).

<img src="resources/mask_step_size.jpg" width="400" align="middle"/>


### <mark>실습</mark> extract_patches
`extract_patches`함수를 완성하세요
- `slide.read_region`함수를 사용하여 원본 이미지로 부터 패치를 추출합니다.

In [ ]:
import random
import os
def extract_patches(slide, slide_thumbnail, tumor_mask, normal_mask, mask_level, patch_config = dict(), save_path = None):
    """
    Extracts normal and tumor patches from the slide using the provided masks.

    Args:
        slide (OpenSlide object): The whole slide image.
        slide_thumbnail (numpy array): The slide thumbnail image.
        tumor_mask (numpy array): The mask of tumor areas.
        normal_mask (numpy array): The mask of normal tissue areas.
        mask_level (int): The level of the mask (resolution level).
        patch_config (dict): Dictionary containing configuration for patch extraction.
        save_path (str, optional): Directory to save extracted patches. Defaults to None.
       """

    patch_size = patch_config.get('patch_size', 304)  # Patch size at the highest resolution
    
    normal_area_threshold = patch_config.get('normal_area_threshold', 0.1) # normal mask inclusion ratio that select normal patches
    normal_sel_ratio = patch_config.get('normal_sel_ratio', 1) # nomral patch selection ratio 
    max_normal_patches = patch_config.get('max_normal_patches', 1000) # number limit of normal patches 

    tumor_area_threshold = patch_config.get('tumor_area_threshold', 0.8) # tumor mask inclusion ratio that select tumor patches
    tumor_sel_ratio = patch_config.get('tumor_sel_ratio', 1) # tumor patch selection ratio
    max_tumor_patches = patch_config.get('max_tumor_patches', 1000) # number limit of tumor patches


    downsample_factor = 2 ** mask_level
    mask_step_size = patch_size // downsample_factor  # Step size at the mask level

    slide_width, slide_height = slide.level_dimensions[0]
    num_patches_x = slide_width // patch_size
    num_patches_y = slide_height // patch_size
    total_patches = num_patches_x * num_patches_y

    tumor_patches_extracted = 0
    normal_patches_extracted = 0
    patches_processed = 0

    if save_path is not None:
        os.makedirs(save_path, exist_ok=True)

    for i in range(num_patches_x):
        for j in range(num_patches_y):
            rand = random.random()

            x_mask = i * mask_step_size
            y_mask = j * mask_step_size
            x_slide = i * patch_size
            y_slide = j * patch_size

            ##### YOUR CODE START #####
            # Extract corresponding mask regions
            tumor_mask_region = None    # TODO
            normal_mask_region = None  # TODO
            mask_area = mask_step_size * mask_step_size * 255
            
            tumor_area_ratio = None    # TODO
            normal_area_ratio = None  # TODO

            # Extract tumor patches
            if (tumor_area_ratio > tumor_area_threshold) and (rand <= tumor_sel_ratio) and (tumor_patches_extracted < max_tumor_patches):
                patch = None # TODO
                if save_path:
                    patch.save(f"{save_path}/t_{str(i)}_{str(j)}.png")
                cv2.rectangle(slide_thumbnail, (x_mask, y_mask), (x_mask + mask_step_size, y_mask + mask_step_size), (0, 0, 255), 2)
                tumor_patches_extracted += 1
            
            # Extract normal patches
            elif (normal_area_ratio > normal_area_threshold) and (tumor_area_ratio == 0) and (rand <= normal_sel_ratio) and (normal_patches_extracted < max_normal_patches):
                patch = None # TODO
                if save_path:
                    patch.save(f"{save_path}/n_{str(i)}_{str(j)}.png")
                cv2.rectangle(slide_thumbnail, (x_mask, y_mask), (x_mask + mask_step_size, y_mask + mask_step_size), (255, 255, 0), 2)
                normal_patches_extracted += 1
            
            patches_processed += 1
            ##### YOUR CODE END #####

    print(f'Processed {patches_processed}/{total_patches} patches.')
    print(f'Extracted {tumor_patches_extracted} tumor patches and {normal_patches_extracted} normal patches.')


def draw_patches(patch, slide_thumbnail_patch, tumor_mask_patch):
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    images = [patch, slide_thumbnail_patch, tumor_mask_patch]
    titles = ['Patch', 'Slide Thumbnail', 'Tumor Mask']

    for ax, img, title in zip(axes.ravel(), images, titles):
        cmap = 'gray' if len(img.shape) == 2 else None  # Use grayscale colormap for 2D masks
        ax.imshow(img, cmap=cmap, vmin=0, vmax=255)
        ax.set_title(title)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
mask_level = 4  # define resolution level of mask

slide, slide_thumbnail, tumor_mask, tissue_mask, normal_mask = create_masks(slide_filepath, annotation_filepath, mask_level = mask_level)
extract_patches(slide, slide_thumbnail, tumor_mask, normal_mask, mask_level = mask_level, save_path = None)

plt.figure(figsize = (16,16))
plt.imshow(slide_thumbnail)
plt.axis('off')
plt.show()

### Clean up saved patches

In [ ]:
import shutil, os
if os.path.exists('patches/'):
    shutil.rmtree('patches/')

# Building a deep learning model
## PatchCamelyon (PCam) 데이터셋

PatchCamelyon 데이터셋은 Camelyon16 데이터셋으로 부터 (96 x 96px) 크기의 patch를 추출해놓은 데이터셋이며, 유방암 환자의 림프절(lymph node) 병리(histopathology) 이미지로 부터 추출된 327,680개의 RGB 컬러 이미지로 구성되어 있습니다.

**Label (Target)**
PCam 데이터셋의 라벨(target)은 0 (normal), 1(tumor)이며, 림프절에서의 암 존재 여부를 분류하는 딥러닝 모델을 개발/평가하는데 사용 될 수 있습니다.
- 만약 patch 중심의 (32 x 32px)영역중 한 개 이상의 pixel이 종양 조직이면 1의 라벨을 가집니다.

In [ ]:
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

from training_utilities import train_loop, evaluation_loop, create_dataloaders, save_checkpoint, load_checkpoint

In [ ]:
def load_pcam_datasets(data_root_dir):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    
    train_transforms = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    train_dataset = datasets.PCAM(root=data_root_dir, split = "train", transform=train_transforms)
    val_dataset = datasets.PCAM(root=data_root_dir, split = "val", transform=test_transforms)
    test_dataset = datasets.PCAM(root=data_root_dir, split = "test", transform=test_transforms)

    num_classes = 2

    return train_dataset, val_dataset, test_dataset, num_classes

PyTorch `Dataset`은 데이터(feature)와 타겟(target, label)을 한번에 하나씩 가져오는 기능을 제공한다.

index를 전달받으면 그 index에 대응하는 데이터(이미지/feature)와 target을 리턴한다.

In [ ]:
data_root_dir = "/datasets"
train_dataset, val_dataset, test_dataset, num_classes = load_pcam_datasets(data_root_dir)

print("Train size: ", len(train_dataset))
print("Validation size: ", len(val_dataset))
print("Test size: ", len(test_dataset))
print("Image shape: ", train_dataset[0][0].shape)
print("Label of fisrt example: ", train_dataset[0][1])

In [ ]:
import matplotlib.pyplot as plt

def visualize_few_samples(dataset, cols=8, rows=5, shuffle = False):
    label_names = ['normal', 'tumor']

    if shuffle:
        sample_indices = np.random.randint(0, len(dataset), size=cols * rows)
    else:
        sample_indices = list(range(cols * rows))

    mean = torch.tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
    
    figure, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2)) 
    axes = axes.flatten()

    for i, sample_idx in enumerate(sample_indices):
        img, label = dataset[sample_idx]
        img = img * std + mean   # Unnormalize to [0,1] for display
        img = img.permute(1, 2, 0).numpy()  # CHW to HWC, to numpy
        axes[i].imshow(img)
        axes[i].set_title(label_names[label])
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_few_samples(train_dataset, cols = 5, rows = 4, shuffle = False)

## stain variation
이미지를 살펴보면 이미지 별로 색상이 차이가 있는것을 확인할 수 있다.

이는 병원마다 H&E 염색을 수행하는 프로토콜이 서로 다르기 때문입니다. 딥러닝 모델을 학습할 때 이러한 변이에 일반화(generalization)할 수 있도록 학습 과정을 설계하는 것이 중요합니다.

1. **Stain normalization methods**

염색 차이를 보정하기 위해 Stain Normalization 기법을 사용할 수 있습니다.

<img src="resources/stain_normalization.png" width="500" align="middle"/>

Macenko's Algorithm : SVD (Singular value decomposition)을 이용해 hematoxylin 채널과 eosin채널을 분리하고, 이를 정규화(normalize) 하는 방법입니다

<img src="resources/stain_normalization_Macenko.jpg" width="700" align="middle"/>

단점
- 특정 기관이나 데이터셋을 대표한다고 여겨지는 참조 이미지(Reference Image)에 의존하므로, 일반화 성능이 떨어질 수 있습니다.
- stain artifact에 불안정할 수 있습니다..

최근에는 보다 안정적인 결과를 얻기 위해 GAN과 같은 딥러닝 기반의 Stain Normalization 방법이 널리 사용되고 있습니다.

2. **Data Augmentation**

모델이 다양한 stain variation에 대해 일반화될 수 있도록, 학습 과정에서 다양한 stain Variation을 가진 이미지에 노출시키는 방법입니다.

- 이번 실습에서는 `ColorJitter`를 사용하여 색상, 밝기, 대비, 채도를 변형하는 Data Augmentation을 수행합니다.
- 단, 데이터가 부족하거나 stain variation을 지나치게 줄 경우 학습이 더 어려워 질 수 있습니다.

In [ ]:
def load_pcam_datasets(data_root_dir):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    
    train_transforms = transforms.Compose([
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        normalize
    ])

    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])

    train_dataset = datasets.PCAM(root=data_root_dir, split = "train", transform=train_transforms)
    val_dataset = datasets.PCAM(root=data_root_dir, split = "val", transform=test_transforms)
    test_dataset = datasets.PCAM(root=data_root_dir, split = "test", transform=test_transforms)

    num_classes = 2

    return train_dataset, val_dataset, test_dataset, num_classes

In [ ]:
train_dataset, val_dataset, test_dataset, num_classes = load_pcam_datasets(data_root_dir)
visualize_few_samples(train_dataset, cols = 5, rows = 4, shuffle = False)

## DataLoader
딥러닝 학습에서는 샘플들을 주로 "mini-batches"로 가져온고, 매 epoch마다 랜덤하게 섞어주며, multiprocessing을 사용해 데이터 획득을 빠르게 합니다.

DataLoader이 복잡한 과정을 쉽도록 도와준다.

- `DataLoader`는 `Dataset`을 배치 단위로 묶어준다 (`batch_size`).
- `shuffle`를 통해 매 epoch마다 랜덤하게 섞어주는 기능을 제공한다
- `num_workers`를 통해 데이터 전처리를 multiprocessing으로 수행할 수 있다.

In [ ]:
batch_size = 32

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers = 2)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}, dtype: {y.dtype}")
    print(f"Target y values: {y}")
    break

In [ ]:
def get_model(model_name, num_classes, config):
    if model_name == "resnet50":
        if config.get('pretrained', ""): #if pretrained model name is given
            print(f'Using pretrained model {config["pretrained"]}')
            model = models.resnet50(weights = config["pretrained"])
            model.fc = nn.Linear(model.fc.in_features, num_classes)
                
        else:
            model = models.resnet50()
            model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == "densenet121":
        if config.get('pretrained', ""): #if pretrained model name is given
            print(f'Using pretrained model {config["pretrained"]}')
            model = models.densenet121(weights = config["pretrained"]) 
        else:
            model = models.densenet121()
        model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    else:
        raise Exception("Model not supported: {}".format(model_name))
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Using model {model_name} with {total_params} parameters ({trainable_params} trainable)")

    return model

## 모델 학습 (training)

In [ ]:
def train_main(config):
    ## data and preprocessing settings
    data_root_dir = config['data_root_dir']
    num_worker = config.get('num_worker', 4)

    ## Hyper parameters
    batch_size = config['batch_size']
    learning_rate = config['learning_rate']
    start_epoch = config.get('start_epoch', 0)
    num_epochs = config['num_epochs']

    ## checkpoint setting
    checkpoint_save_interval = config.get('checkpoint_save_interval', 10)
    checkpoint_path = config.get('checkpoint_path', "checkpoints/checkpoint.pth")
    best_model_path = config.get('best_model_path', "checkpoints/best_model.pth")
    load_from_checkpoint = config.get('load_from_checkpoint', None)

    ## variables
    best_acc1 = 0

    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using {device} device")

    train_dataset, val_dataset, test_dataset, num_classes = load_pcam_datasets(data_root_dir)
    
    train_dataloader, val_dataloader, test_dataloader = create_dataloaders(train_dataset, val_dataset, test_dataset, device, 
                                                           batch_size = batch_size, num_worker = num_worker)
        
    model = get_model(model_name = config["model_name"], num_classes= num_classes, config = config).to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1) 

    if load_from_checkpoint:
        load_checkpoint_path = (best_model_path if load_from_checkpoint == "best" else checkpoint_path)
        start_epoch, best_acc1 = load_checkpoint(load_checkpoint_path, model, optimizer, scheduler, device)

    if config.get('test_mode', False):
        # Only evaluate on the test dataset
        print("Running test evaluation...")
        test_acc = evaluation_loop(model, device, test_dataloader, criterion, phase = "test")
        print(f"Test Accuracy: {test_acc}")
        
    else:
        # Train and validate using train/val datasets
        for epoch in range(start_epoch, num_epochs):
            train_loop(model, device, train_dataloader, criterion, optimizer, epoch)
            val_acc1 = evaluation_loop(model, device, val_dataloader, criterion, epoch = epoch, phase = "validation")
            scheduler.step()

            if (epoch + 1) % checkpoint_save_interval == 0 or (epoch + 1) == num_epochs:
                is_best = val_acc1 > best_acc1
                best_acc1 = max(val_acc1, best_acc1)
                save_checkpoint(checkpoint_path, model, optimizer, scheduler, epoch, best_acc1, is_best, best_model_path)

In [ ]:
config = {
    'data_root_dir': '/datasets',
    'batch_size': 256,
    'learning_rate': 1e-3,
    'num_epochs': 1,
    'model_name': 'densenet121',
    'pretrained' : 'IMAGENET1K_V1',

    "checkpoint_save_interval" : 1,
    "checkpoint_path" : "checkpoints/checkpoint.pth",
    "best_model_path" : "checkpoints/best_model.pth",
    "load_from_checkpoint" : None,    # Options: "latest", "best", or None
}
train_main(config)


In [ ]:
config_testmode = {
    **config, 
    'test_mode': True, # True if evaluating only test set
    'load_from_checkpoint': 'best'
}

train_main(config_testmode)

### <mark>실습</mark> .gitignore
모델 checkpoint를 git에 올리지 않기 위에 `.gitignore`를 수정하세요.

### Lab을 마무리 짓기 전 저장된 checkpoint를 모두 지워 저장공간을 확보한다

In [ ]:
import shutil, os
if os.path.exists('checkpoints/'):
    shutil.rmtree('checkpoints/')